!pip3 uninstall -y torch torchvision

!pip3 install torch torchvision

In [3]:
import torch
# import the data
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import RandomOverSampler

In [4]:
import os
os.environ["KERAS_BACKEND"] = "torch"

In [5]:
import numpy as np
import keras
from keras import layers

In [6]:
# function to evaluate model
# returns (accuracy, tpr, tnr, f1)
# takes as input real, predicted
def genResults(real, predicted):
    real=np.argmax(real, axis=1)
    predicted=np.argmax(predicted, axis=1)

    conf_matrix = confusion_matrix(y_true=real, y_pred=predicted)
    tn, fp, fn, tp = conf_matrix.ravel()

    f1 = (2*tp)/((2*tp)+fp+fn)

    acc = (tp + tn) / (tp + tn + fn + fp)

    tpr = tp / (tp + fp)
    tnr = tn / (tn + fn)

    print("true negative: ", tn)
    print("false positive: ", fp)
    print("false negative: ", fn)
    print("true positive: ", tp)
    
    return (acc, tpr, tnr, f1)

In [7]:
# split it like above

# do the shaping if necessary
# change the type of the columns that are characters to one-hot-encodings or something like that

# make a model with one layer
# run the model and see what happens

In [8]:
df = pd.read_csv('data\\online_shoppers_intention.csv') #TODO - change to new dataset

print(df)

# hyperparameters
batch_size = 128
epochs = 30
dropOut = 0.5

       Administrative  Administrative_Duration  Informational  \
0                   0                      0.0              0   
1                   0                      0.0              0   
2                   0                      0.0              0   
3                   0                      0.0              0   
4                   0                      0.0              0   
...               ...                      ...            ...   
12325               3                    145.0              0   
12326               0                      0.0              0   
12327               0                      0.0              0   
12328               4                     75.0              0   
12329               0                      0.0              0   

       Informational_Duration  ProductRelated  ProductRelated_Duration  \
0                         0.0               1                 0.000000   
1                         0.0               2                64.000000 

In [9]:
df['total_duration'] = df['Administrative_Duration'] + df['Informational_Duration'] + df['ProductRelated_Duration']
df['proportion_of_duration'] = df['ProductRelated_Duration'] / df['total_duration']

In [10]:
df.drop(columns=['Informational_Duration', 'Administrative_Duration', 
                 'ProductRelated_Duration', 'Administrative', 'Informational', 
                 'ProductRelated', 'OperatingSystems', 'Browser'])

#df = df.drop(columns=['Informational_Duration', 'Administrative_Duration', 
#                 'ProductRelated_Duration', 'Administrative', 'Informational', 
#                 'ProductRelated', 'OperatingSystems', 'Browser',
#                 'BounceRates', 'ExitRates', 'SpecialDay', 'Month',
#                 'Region', 'TrafficType', 'VisitorType', 'Weekend'])

,BounceRates,ExitRates,PageValues,SpecialDay,Month,Region,TrafficType,VisitorType,Weekend,Revenue,total_duration,proportion_of_duration
0,0.200000,0.200000,0.000000,0.0,Feb,1,1,Returning_Visitor,False,False,0.000000,NaN
1,0.000000,0.100000,0.000000,0.0,Feb,1,2,Returning_Visitor,False,False,64.000000,1.000000
2,0.200000,0.200000,0.000000,0.0,Feb,9,3,Returning_Visitor,False,False,0.000000,NaN
3,0.050000,0.140000,0.000000,0.0,Feb,2,4,Returning_Visitor,False,False,2.666667,1.000000
4,0.020000,0.050000,0.000000,0.0,Feb,1,4,Returning_Visitor,True,False,627.500000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
12325,0.007143,0.029031,12.241717,0.0,Dec,1,1,Returning_Visitor,True,False,1928.791667,0.924823
12326,0.000000,0.021333,0.000000,0.0,Nov,1,8,Returning_Visitor,True,False,465.750000,1.000000
12327,0.083333,0.086667,0.000000,0.0,Nov,1,13,Returning_Visitor,True,False,184.250000,1.000000
12328,0.000000,0.021053,0.000000,0.0,Nov,3,11,Returning_Visitor,False,False,421.000000,0.821853


In [11]:
# df['Month'].replace(['
#print(df['Month'].unique())

In [12]:
df['Month'].replace(['Jan','Feb','Mar','Apr','May','June','Jul','Aug','Sep','Oct','Nov','Dec'],
                    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], inplace=True)

C:\Users\tomli\AppData\Local\Temp\ipykernel_16360\727251348.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Month'].replace(['Jan','Feb','Mar','Apr','May','June','Jul','Aug','Sep','Oct','Nov','Dec'],
C:\Users\tomli\AppData\Local\Temp\ipykernel_16360\727251348.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_dow

In [13]:
#print(df['Month'].unique())

In [14]:
#print(df['VisitorType'].unique())

In [15]:
df['VisitorType'].replace(['Returning_Visitor', 'New_Visitor', 'Other'], [0, 1, 2], inplace=True)

C:\Users\tomli\AppData\Local\Temp\ipykernel_16360\656729720.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['VisitorType'].replace(['Returning_Visitor', 'New_Visitor', 'Other'], [0, 1, 2], inplace=True)
C:\Users\tomli\AppData\Local\Temp\ipykernel_16360\656729720.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_d

In [16]:
df_copy = df.copy(deep=True)
print (df)

       Administrative  Administrative_Duration  Informational  \
0                   0                      0.0              0   
1                   0                      0.0              0   
2                   0                      0.0              0   
3                   0                      0.0              0   
4                   0                      0.0              0   
...               ...                      ...            ...   
12325               3                    145.0              0   
12326               0                      0.0              0   
12327               0                      0.0              0   
12328               4                     75.0              0   
12329               0                      0.0              0   

       Informational_Duration  ProductRelated  ProductRelated_Duration  \
0                         0.0               1                 0.000000   
1                         0.0               2                64.000000 

In [17]:
df_copy['Weekend'].replace([False, True], [0,1], inplace=True)

C:\Users\tomli\AppData\Local\Temp\ipykernel_16360\2402203070.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_copy['Weekend'].replace([False, True], [0,1], inplace=True)
C:\Users\tomli\AppData\Local\Temp\ipykernel_16360\2402203070.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_copy['We

In [18]:
df_copy['Revenue'].replace([False, True], [0,1], inplace=True)

C:\Users\tomli\AppData\Local\Temp\ipykernel_16360\550847130.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_copy['Revenue'].replace([False, True], [0,1], inplace=True)
C:\Users\tomli\AppData\Local\Temp\ipykernel_16360\550847130.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_copy['Reve

In [19]:
df_copy = df_copy.drop(columns=['Informational_Duration', 'Administrative_Duration', 
                 'ProductRelated_Duration', 'Administrative', 'Informational', 
                 'ProductRelated', 'OperatingSystems', 'Browser'])

In [20]:
print(df_copy)

       BounceRates  ExitRates  PageValues  SpecialDay  Month  Region  \
0         0.200000   0.200000    0.000000         0.0      2       1   
1         0.000000   0.100000    0.000000         0.0      2       1   
2         0.200000   0.200000    0.000000         0.0      2       9   
3         0.050000   0.140000    0.000000         0.0      2       2   
4         0.020000   0.050000    0.000000         0.0      2       1   
...            ...        ...         ...         ...    ...     ...   
12325     0.007143   0.029031   12.241717         0.0     12       1   
12326     0.000000   0.021333    0.000000         0.0     11       1   
12327     0.083333   0.086667    0.000000         0.0     11       1   
12328     0.000000   0.021053    0.000000         0.0     11       3   
12329     0.000000   0.066667    0.000000         0.0     11       1   

       TrafficType  VisitorType  Weekend  Revenue  total_duration  \
0                1            0        0        0        0.000000 

In [21]:
df_copy.fillna(0, inplace=True)

In [22]:
df_numpy = df_copy.to_numpy()

In [23]:
df_numpy.shape[0]
testSample = np.random.choice(df_numpy.shape[0], math.floor(0.7 * df_numpy.shape[0]), replace=False)

In [24]:
print(testSample)

[3318 1478 3425 ...  336 5075 2203]


In [25]:
print(df_numpy[testSample])

train = df_numpy[testSample]
test = df_numpy[~testSample]

print("Train length: " + str(train.shape[0]))
print("Test lenght: " + str(test.shape[0]))

[[0.00000000e+00 5.00000000e-03 0.00000000e+00 ... 0.00000000e+00
  2.27666667e+02 5.60761347e-01]
 [0.00000000e+00 2.22222220e-02 0.00000000e+00 ... 0.00000000e+00
  6.85000000e+02 1.00000000e+00]
 [3.62500000e-02 4.73958330e-02 1.49925000e+01 ... 1.00000000e+00
  4.22000000e+02 9.40758294e-01]
 ...
 [0.00000000e+00 2.85714290e-02 0.00000000e+00 ... 0.00000000e+00
  1.26848571e+03 9.74547830e-01]
 [3.87409200e-03 2.33615820e-02 2.69379237e+01 ... 0.00000000e+00
  1.95277460e+03 9.36500608e-01]
 [4.90956100e-03 2.08878900e-02 1.02122977e+01 ... 0.00000000e+00
  4.35202576e+03 8.92004316e-01]]
Train length: 8631
Test lenght: 8631


In [26]:
df_copy_y = df_copy['Revenue']
df_copy_x = df_copy.loc[:, df_copy.columns != 'Revenue']

print(df_copy_y)
print(df_copy_x)

0        0
1        0
2        0
3        0
4        0
        ..
12325    0
12326    0
12327    0
12328    0
12329    0
Name: Revenue, Length: 12330, dtype: int64
       BounceRates  ExitRates  PageValues  SpecialDay  Month  Region  \
0         0.200000   0.200000    0.000000         0.0      2       1   
1         0.000000   0.100000    0.000000         0.0      2       1   
2         0.200000   0.200000    0.000000         0.0      2       9   
3         0.050000   0.140000    0.000000         0.0      2       2   
4         0.020000   0.050000    0.000000         0.0      2       1   
...            ...        ...         ...         ...    ...     ...   
12325     0.007143   0.029031   12.241717         0.0     12       1   
12326     0.000000   0.021333    0.000000         0.0     11       1   
12327     0.083333   0.086667    0.000000         0.0     11       1   
12328     0.000000   0.021053    0.000000         0.0     11       3   
12329     0.000000   0.066667    0.000000   

In [27]:
df_copy_x_train, df_copy_x_test, df_copy_y_train, df_copy_y_test = train_test_split(
    df_copy_x, df_copy_y, test_size=0.3, random_state=42)

In [28]:
numpy_x_train = df_copy_x_train.to_numpy()
numpy_x_test = df_copy_x_test.to_numpy()
numpy_y_train = df_copy_y_train.to_numpy()
numpy_y_test = df_copy_y_test.to_numpy()

num_classes = 2

input_shape = (11, 1)

# Make sure images have shape (28, 28, 1)
numpy_x_train = np.expand_dims(numpy_x_train, -1)
numpy_x_test = np.expand_dims(numpy_x_test, -1)
print("x_train shape:", numpy_x_train.shape)
print(numpy_x_train.shape[0], "train samples")
print(numpy_x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
numpy_y_train = keras.utils.to_categorical(numpy_y_train, num_classes)
numpy_y_test = keras.utils.to_categorical(numpy_y_test, num_classes)

print("y_train shape: ", numpy_y_train.shape)

x_train shape: (8631, 11, 1)
8631 train samples
3699 test samples
y_train shape:  (8631, 2)


In [29]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Dense(10, activation="relu"),
        layers.Flatten(),
        layers.Dropout(dropOut),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 11, 10)              │              20 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 110)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 110)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             222 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 242 (968.00 B)

 Trainable params: 242 (968.00 B)

 Non-trainable params: 0 (0.00 B)

In [30]:
model.compile(loss="mean_squared_error", optimizer="adam", 
              metrics=["accuracy"])

model.fit(numpy_x_train, numpy_y_train, batch_size=batch_size, 
          epochs=epochs, validation_split=0.1)

Epoch 1/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3826 - loss: 0.6011 - val_accuracy: 0.5648 - val_loss: 0.3791
Epoch 2/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6349 - loss: 0.3589 - val_accuracy: 0.8472 - val_loss: 0.1530
Epoch 3/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7244 - loss: 0.2706 - val_accuracy: 0.8461 - val_loss: 0.1539
Epoch 4/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8188 - loss: 0.1794 - val_accuracy: 0.8461 - val_loss: 0.1539
Epoch 5/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8245 - loss: 0.1740 - val_accuracy: 0.8461 - val_loss: 0.1539
Epoch 6/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8554 - loss: 0.1441 - val_accuracy: 0.8461 - val_loss: 0.1536
Epoch 7/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8446 - loss: 0.1543 - val_accuracy: 0.8461 - val_loss: 0.1535
Epoch 8/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8554 - loss: 0.1440 - val_accuracy: 0.8472 - val_loss:

In [31]:
predicted = model.predict(numpy_x_test)

116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [32]:
print("numpy_y_test shape: ", numpy_y_test.shape)
print(numpy_y_test)
print("predicted: ", predicted.shape)
print(predicted)

numpy_y_test shape:  (3699, 2)
[[1. 0.]
 [0. 1.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]
predicted:  (3699, 2)
[[1.00000000e+00 1.05502252e-15]
 [1.00000000e+00 5.26568572e-33]
 [1.00000000e+00 8.40779079e-45]
 ...
 [9.99881983e-01 1.18048585e-04]
 [3.11749894e-16 1.00000000e+00]
 [1.00000000e+00 3.08029519e-10]]


In [33]:
acc, tpr, tnr, f1 = genResults(numpy_y_test, predicted)

print("10 layers:")
print("Accuracy: ", acc)
print("True-positive rate: ", tpr)
print("True-negative rate: ", tnr)
print("F1 Score: ", f1)

true negative:  3067
false positive:  57
false negative:  368
true positive:  207
10 layers:
Accuracy:  0.8851040821843742
True-positive rate:  0.7840909090909091
True-negative rate:  0.8928675400291121
F1 Score:  0.4934445768772348


In [34]:
model20 = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Dense(20, activation="relu"),
        layers.Flatten(),
        layers.Dropout(dropOut),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model20.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                      │ (None, 11, 20)              │              40 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 220)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 220)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 2)                   │             442 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 482 (1.88 KB)

 Trainable params: 482 (1.88 KB)

 Non-trainable params: 0 (0.00 B)

In [35]:
model20.compile(loss="mean_squared_error", optimizer="adam", 
              metrics=["accuracy"])

model20.fit(numpy_x_train, numpy_y_train, batch_size=batch_size, 
          epochs=epochs, validation_split=0.1)

Epoch 1/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1943 - loss: 0.7881 - val_accuracy: 0.2593 - val_loss: 0.7245
Epoch 2/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6152 - loss: 0.3771 - val_accuracy: 0.8461 - val_loss: 0.1540
Epoch 3/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8470 - loss: 0.1518 - val_accuracy: 0.8461 - val_loss: 0.1539
Epoch 4/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8510 - loss: 0.1485 - val_accuracy: 0.8461 - val_loss: 0.1539
Epoch 5/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8444 - loss: 0.1551 - val_accuracy: 0.8461 - val_loss: 0.1539
Epoch 6/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8434 - loss: 0.1562 - val_accuracy: 0.8461 - val_loss: 0.1539
Epoch 7/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8412 - loss: 0.1583 - val_accuracy: 0.8461 - val_loss: 0.1539
Epoch 8/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8454 - loss: 0.1541 - val_accuracy: 0.8461 - val_loss:

In [36]:
predicted20 = model20.predict(numpy_x_test)

116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [37]:
print("numpy_y_test shape: ", numpy_y_test.shape)
print(numpy_y_test)
print("predicted: ", predicted20.shape)
print(predicted)

numpy_y_test shape:  (3699, 2)
[[1. 0.]
 [0. 1.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]
predicted:  (3699, 2)
[[1.00000000e+00 1.05502252e-15]
 [1.00000000e+00 5.26568572e-33]
 [1.00000000e+00 8.40779079e-45]
 ...
 [9.99881983e-01 1.18048585e-04]
 [3.11749894e-16 1.00000000e+00]
 [1.00000000e+00 3.08029519e-10]]


In [38]:
acc20, tpr20, tnr20, f120 = genResults(numpy_y_test, predicted20)

print("20 layers:")
print("Accuracy: ", acc20)
print("True-positive rate: ", tpr20)
print("True-negative rate: ", tnr20)
print("F1 Score: ", f120)

true negative:  3053
false positive:  71
false negative:  350
true positive:  225
20 layers:
Accuracy:  0.8861854555285212
True-positive rate:  0.7601351351351351
True-negative rate:  0.8971495739053776
F1 Score:  0.5166475315729047


In [39]:
model40 = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Dense(40, activation="relu"),
        layers.Flatten(),
        layers.Dropout(dropOut),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model40.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 11, 40)              │              80 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 440)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 440)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 2)                   │             882 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 962 (3.76 KB)

 Trainable params: 962 (3.76 KB)

 Non-trainable params: 0 (0.00 B)

In [40]:
model40.compile(loss="mean_squared_error", optimizer="adam", 
              metrics=["accuracy"])

model40.fit(numpy_x_train, numpy_y_train, batch_size=batch_size, 
          epochs=epochs, validation_split=0.1)

Epoch 1/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6464 - loss: 0.3428 - val_accuracy: 0.8461 - val_loss: 0.1544
Epoch 2/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8377 - loss: 0.1617 - val_accuracy: 0.8461 - val_loss: 0.1541
Epoch 3/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8481 - loss: 0.1518 - val_accuracy: 0.8461 - val_loss: 0.1540
Epoch 4/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8487 - loss: 0.1513 - val_accuracy: 0.8461 - val_loss: 0.1540
Epoch 5/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8453 - loss: 0.1546 - val_accuracy: 0.8461 - val_loss: 0.1539
Epoch 6/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8406 - loss: 0.1590 - val_accuracy: 0.8461 - val_loss: 0.1536
Epoch 7/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8417 - loss: 0.1582 - val_accuracy: 0.8472 - val_loss: 0.1529
Epoch 8/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8499 - loss: 0.1499 - val_accuracy: 0.8461 - val_loss:

In [41]:
predicted40 = model40.predict(numpy_x_test)

116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [42]:
acc40, tpr40, tnr40, f140 = genResults(numpy_y_test, predicted40)

print("40 layers:")
print("Accuracy: ", acc40)
print("True-positive rate: ", tpr40)
print("True-negative rate: ", tnr40)
print("F1 Score: ", f140)

true negative:  2953
false positive:  171
false negative:  249
true positive:  326
40 layers:
Accuracy:  0.8864557988645579
True-positive rate:  0.6559356136820925
True-negative rate:  0.9222361024359775
F1 Score:  0.6082089552238806


In [43]:
# now we try oversampling
ros = RandomOverSampler(random_state=42)
df_copy_x_resampled, df_copy_y_resampled = ros.fit_resample(df_copy_x, df_copy_y)

df_copy_x_resampled_train, df_copy_x__resampled_test, df_copy_y_resampled_train, df_copy_y_resampled_test = train_test_split(
    df_copy_x_resampled, df_copy_y_resampled, test_size=0.3, random_state=42)

In [44]:
numpy_x_resampled_train = df_copy_x_resampled_train.to_numpy()
numpy_x_resampled_test = df_copy_x__resampled_test.to_numpy()
numpy_y_resampled_train = df_copy_y_resampled_train.to_numpy()
numpy_y_resampled_test = df_copy_y_resampled_test.to_numpy()

num_classes = 2

# input_shape = (2, 1)

# Make sure images have shape (28, 28, 1)
numpy_x_resampled_train = np.expand_dims(numpy_x_resampled_train, -1)
numpy_x_resampled_test = np.expand_dims(numpy_x_resampled_test, -1)
print("x_train shape:", numpy_x_resampled_train.shape)
print(numpy_x_resampled_train.shape[0], "train samples")
print(numpy_x_resampled_test.shape[0], "test samples")

# convert class vectors to binary class matrices
numpy_y_resampled_train = keras.utils.to_categorical(numpy_y_resampled_train, num_classes)
numpy_y_resampled_test = keras.utils.to_categorical(numpy_y_resampled_test, num_classes)

print("y_train shape: ", numpy_y_resampled_train.shape)

x_train shape: (14590, 11, 1)
14590 train samples
6254 test samples
y_train shape:  (14590, 2)


In [45]:
model_r = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Dense(10, activation="relu"),
        layers.Flatten(),
        layers.Dropout(dropOut),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model_r.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                      │ (None, 11, 10)              │              20 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 110)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 110)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 2)                   │             222 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 242 (968.00 B)

 Trainable params: 242 (968.00 B)

 Non-trainable params: 0 (0.00 B)

In [46]:
model_r.compile(loss="mean_squared_error", optimizer="adam", 
              metrics=["accuracy"])

model_r.fit(numpy_x_resampled_train, numpy_y_resampled_train, batch_size=batch_size, 
          epochs=epochs, validation_split=0.1)

Epoch 1/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5021 - loss: 0.4754 - val_accuracy: 0.5161 - val_loss: 0.4757
Epoch 2/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5395 - loss: 0.4472 - val_accuracy: 0.5579 - val_loss: 0.4285
Epoch 3/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5961 - loss: 0.3698 - val_accuracy: 0.6422 - val_loss: 0.3339
Epoch 4/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6888 - loss: 0.2745 - val_accuracy: 0.8081 - val_loss: 0.1566
Epoch 5/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7539 - loss: 0.1978 - val_accuracy: 0.8321 - val_loss: 0.1301
Epoch 6/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7604 - loss: 0.1885 - val_accuracy: 0.7471 - val_loss: 0.1952
Epoch 7/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7492 - loss: 0.1980 - val_accuracy: 0.8129 - val_loss: 0.1507
Epoch 8/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7616 - loss: 0.1839 - val_accuracy: 0.

In [47]:
predicted_r = model_r.predict(numpy_x_resampled_test)

196/196 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [48]:
acc_r, tpr_r, tnr_r, f1_r = genResults(numpy_y_resampled_test, predicted_r)

print("10 layers:")
print("Accuracy: ", acc_r)
print("True-positive rate: ", tpr_r)
print("True-negative rate: ", tnr_r)
print("F1 Score: ", f1_r)

true negative:  2759
false positive:  364
false negative:  654
true positive:  2477
10 layers:
Accuracy:  0.8372241765270227
True-positive rate:  0.8718760999648011
True-negative rate:  0.8083797245824788
F1 Score:  0.8295378432685867


In [49]:
model_r_20 = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Dense(20, activation="relu"),
        layers.Flatten(),
        layers.Dropout(dropOut),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model_r_20.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                      │ (None, 11, 20)              │              40 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 220)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 220)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 2)                   │             442 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 482 (1.88 KB)

 Trainable params: 482 (1.88 KB)

 Non-trainable params: 0 (0.00 B)

In [50]:
model_r_20.compile(loss="mean_squared_error", optimizer="adam", 
              metrics=["accuracy"])

model_r_20.fit(numpy_x_resampled_train, numpy_y_resampled_train, batch_size=batch_size, 
          epochs=epochs, validation_split=0.1)

Epoch 1/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5387 - loss: 0.4546 - val_accuracy: 0.5326 - val_loss: 0.4599
Epoch 2/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5623 - loss: 0.4333 - val_accuracy: 0.5675 - val_loss: 0.4260
Epoch 3/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5856 - loss: 0.4094 - val_accuracy: 0.5949 - val_loss: 0.4012
Epoch 4/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5959 - loss: 0.3993 - val_accuracy: 0.5894 - val_loss: 0.4076
Epoch 5/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6109 - loss: 0.3853 - val_accuracy: 0.6271 - val_loss: 0.3694
Epoch 6/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6280 - loss: 0.3683 - val_accuracy: 0.6758 - val_loss: 0.3227
Epoch 7/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6325 - loss: 0.3644 - val_accuracy: 0.7471 - val_loss: 0.2463
Epoch 8/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6353 - loss: 0.3614 - val_accuracy: 0.

In [51]:
predicted_r_20 = model_r_20.predict(numpy_x_resampled_test)

196/196 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [52]:
acc_r_20, tpr_r_20, tnr_r_20, f1_r_20 = genResults(numpy_y_resampled_test, predicted_r_20)

print("20 layers:")
print("Accuracy: ", acc_r_20)
print("True-positive rate: ", tpr_r_20)
print("True-negative rate: ", tnr_r_20)
print("F1 Score: ", f1_r_20)

true negative:  2279
false positive:  844
false negative:  406
true positive:  2725
20 layers:
Accuracy:  0.8001279181323953
True-positive rate:  0.7635191930512749
True-negative rate:  0.8487895716945997
F1 Score:  0.8134328358208955


In [53]:
model_r_40 = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Dense(40, activation="relu"),
        layers.Flatten(),
        layers.Dropout(dropOut),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model_r_40.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                     │ (None, 11, 40)              │              80 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_5 (Flatten)                  │ (None, 440)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 440)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 2)                   │             882 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 962 (3.76 KB)

 Trainable params: 962 (3.76 KB)

 Non-trainable params: 0 (0.00 B)

In [54]:
model_r_40.compile(loss="mean_squared_error", optimizer="adam", 
              metrics=["accuracy"])

model_r_40.fit(numpy_x_resampled_train, numpy_y_resampled_train, batch_size=batch_size, 
          epochs=epochs, validation_split=0.1)

Epoch 1/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5143 - loss: 0.4702 - val_accuracy: 0.5689 - val_loss: 0.4152
Epoch 2/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5818 - loss: 0.4081 - val_accuracy: 0.6840 - val_loss: 0.3006
Epoch 3/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6193 - loss: 0.3739 - val_accuracy: 0.5908 - val_loss: 0.4039
Epoch 4/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6359 - loss: 0.3574 - val_accuracy: 0.6388 - val_loss: 0.3563
Epoch 5/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6768 - loss: 0.3176 - val_accuracy: 0.6936 - val_loss: 0.2986
Epoch 6/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6937 - loss: 0.3008 - val_accuracy: 0.7786 - val_loss: 0.2113
Epoch 7/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7302 - loss: 0.2642 - val_accuracy: 0.7505 - val_loss: 0.2449
Epoch 8/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7374 - loss: 0.2567 - val_accuracy: 0.

In [55]:
predicted_r_40 = model_r_40.predict(numpy_x_resampled_test)

196/196 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [56]:
acc_r_40, tpr_r_40, tnr_r_40, f1_r_40 = genResults(numpy_y_resampled_test, predicted_r_40)

print("40 layers:")
print("Accuracy: ", acc_r_40)
print("True-positive rate: ", tpr_r_40)
print("True-negative rate: ", tnr_r_40)
print("F1 Score: ", f1_r_40)

true negative:  2339
false positive:  784
false negative:  410
true positive:  2721
40 layers:
Accuracy:  0.809082187400064
True-positive rate:  0.7763195435092725
True-negative rate:  0.8508548563113859
F1 Score:  0.8200723327305606


In [57]:
# Now we need to repeat it all - starting with 20 nodes I think, but with a much smaller fraction
small_x_resampled_train, small_x_resampled_test, small_y_resampled_train, small_y_resampled_test = train_test_split(
    df_copy_x_resampled, df_copy_y_resampled, test_size=0.7, random_state=42)

numpy_small_x_train = small_x_resampled_train.to_numpy()
numpy_small_x_test = small_x_resampled_test.to_numpy()
numpy_small_y_train = small_y_resampled_train.to_numpy()
numpy_small_y_test = small_y_resampled_test.to_numpy()

num_classes = 2

# input_shape = (2, 1)

# Make sure images have shape (11, 1)
numpy_small_x_train = np.expand_dims(numpy_small_x_train, -1)
numpy_small_x_test = np.expand_dims(numpy_small_x_test, -1)
print("x_train shape:", numpy_small_x_train.shape)
print(numpy_small_x_train.shape[0], "train samples")
print(numpy_small_x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
numpy_small_y_train = keras.utils.to_categorical(numpy_small_y_train, num_classes)
numpy_small_y_test = keras.utils.to_categorical(numpy_small_y_test, num_classes)

print("y_train shape: ", numpy_small_y_train.shape)

x_train shape: (6253, 11, 1)
6253 train samples
14591 test samples
y_train shape:  (6253, 2)


In [58]:
model_small_r_20 = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Dense(20, activation="relu"),
        layers.Flatten(),
        layers.Dropout(dropOut),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model_small_r_20.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                     │ (None, 11, 20)              │              40 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_6 (Flatten)                  │ (None, 220)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 220)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 2)                   │             442 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 482 (1.88 KB)

 Trainable params: 482 (1.88 KB)

 Non-trainable params: 0 (0.00 B)

In [59]:
model_small_r_20.compile(loss="mean_squared_error", optimizer="adam", 
              metrics=["accuracy"])

model_small_r_20.fit(numpy_small_x_train, numpy_small_y_train, batch_size=batch_size, 
          epochs=epochs, validation_split=0.1)

Epoch 1/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5285 - loss: 0.4680 - val_accuracy: 0.5495 - val_loss: 0.4510
Epoch 2/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5410 - loss: 0.4518 - val_accuracy: 0.5527 - val_loss: 0.4473
Epoch 3/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5422 - loss: 0.4513 - val_accuracy: 0.5527 - val_loss: 0.4465
Epoch 4/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5226 - loss: 0.4728 - val_accuracy: 0.5639 - val_loss: 0.4351
Epoch 5/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5486 - loss: 0.4461 - val_accuracy: 0.5735 - val_loss: 0.4257
Epoch 6/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5610 - loss: 0.4334 - val_accuracy: 0.5895 - val_loss: 0.4098
Epoch 7/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5758 - loss: 0.4164 - val_accuracy: 0.5895 - val_loss: 0.4065
Epoch 8/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5904 - loss: 0.4055 - val_accuracy: 0.5895 - val_loss:

In [60]:
predicted_r_small_20 = model_small_r_20.predict(numpy_small_x_test)

456/456 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


In [61]:
print(small_y_resampled_test.shape)
print(predicted_r_small_20.shape)

(14591,)
(14591, 2)


In [62]:
acc_r_small_20, tpr_r_small_20, tnr_r_small_20, f1_r_small_20 = genResults(numpy_small_y_test, predicted_r_small_20)

print("Small resampled 20 layers:")
print("Accuracy: ", acc_r_small_20)
print("True-positive rate: ", tpr_r_small_20)
print("True-negative rate: ", tnr_r_small_20)
print("F1 Score: ", f1_r_small_20)

true negative:  4799
false positive:  2512
false negative:  856
true positive:  6424
Small resampled 20 layers:
Accuracy:  0.7691727777397026
True-positive rate:  0.7188898836168308
True-negative rate:  0.8486295313881521
F1 Score:  0.7923038973852985
